In [3]:
!python.exe -m pip install --upgrade pip
!pip install SPARQLWrapper
!pip install rdflib

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/ef/7d/500c9ad20238fcfcb4cb9243eede163594d7020ce87bd9610c9e02771876/pip-24.3.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 653.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.8 MB 1.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/1.8 MB 1.1 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.8 MB 1.3 MB/s eta 0:00:02
   ------- -------------------------------- 0.4/1.8 MB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/1.8 MB 1.7 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 2.1 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.8 MB 2.0 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.8 MB 2.4 M

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.request
import tkinter as tk
from tkinter import scrolledtext, messagebox

def test_https():
    """Test HTTPS connectivity to the SPARQL endpoint."""
    try:
        urllib.request.urlopen("https://dbpedia.org/sparql")
        return True
    except Exception as e:
        messagebox.showerror("HTTPS Error", f"HTTPS failed: {e}")
        return False

def query_dbpedia(endpoint_url, query):
    """Execute a SPARQL query against the given endpoint."""
    endpoint = SPARQLWrapper(endpoint_url)
    endpoint.setQuery(query)
    endpoint.setReturnFormat(JSON)
    try:
        results = endpoint.query().convert()
        return results
    except Exception as e:
        messagebox.showerror("SPARQL Error", f"An error occurred: {e}")
        return None

def process_results(results):
    """Process and display SPARQL query results."""
    if not results:
        return "No results to process."
    
    result_text = "Results:\n"
    for result in results["results"]["bindings"]:
        for key, value in result.items():
            result_text += f"{key}: {value['value']}\n"
        result_text += "---\n"
    return result_text

def on_execute_query(query_name, query, result_display):
    """Handle executing a query and displaying results."""
    if not test_https():
        return
    results = query_dbpedia("https://dbpedia.org/sparql", query)
    result_text = process_results(results)
    result_display.delete(1.0, tk.END)  # Clear previous results
    result_display.insert(tk.END, result_text)  # Display new results

def on_add_query(query_name_entry, query_text_entry, queries, query_menu):
    """Add a new query dynamically."""
    query_name = query_name_entry.get()
    query_text = query_text_entry.get("1.0", tk.END).strip()

    if query_name and query_text:
        queries[query_name] = query_text
        # Update the OptionMenu with the new query
        query_menu['menu'].add_command(label=query_name, command=lambda value=query_name: query_menu.setvar(query_menu.cget("textvariable"), value))
        messagebox.showinfo("Success", f"Query '{query_name}' added successfully!")
    else:
        messagebox.showerror("Error", "Please provide both a query name and the query text.")

def on_delete_query(selected_query, queries, query_menu):
    """Delete the selected query dynamically."""
    query_name = selected_query.get()
    if query_name in queries:
        del queries[query_name]
        # Update the OptionMenu to remove the deleted query
        menu = query_menu['menu']
        menu.delete(menu.index(query_name))  # Remove query from the dropdown menu
        messagebox.showinfo("Success", f"Query '{query_name}' deleted successfully!")
    else:
        messagebox.showerror("Error", "Selected query not found.")

def create_gui():
    """Create the GUI for the application."""
    # Root window
    root = tk.Tk()
    root.title("SPARQL Query Executor")

    # Frame for buttons and input
    frame = tk.Frame(root)
    frame.pack(pady=10)

    # Dropdown for selecting a query
    query_options = [
        "Java programming language",
        "Python programming language",
        "Countries and capitals"
    ]
    selected_query = tk.StringVar(value=query_options[0])
    query_menu = tk.OptionMenu(frame, selected_query, *query_options)
    query_menu.pack()

    # Display area for the results
    result_display = scrolledtext.ScrolledText(root, width=80, height=20)
    result_display.pack(pady=10)

    # Queries dictionary
    queries = {
        "Java programming language": """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?label ?abstract
        WHERE {
            <http://dbpedia.org/resource/Java_(programming_language)> rdfs:label ?label ;
                                                                    dbo:abstract ?abstract . 
            FILTER (lang(?label) = 'en' && lang(?abstract) = 'en')
        }
        LIMIT 10
        """,
        "Python programming language": """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?label ?abstract
        WHERE {
            <http://dbpedia.org/resource/Python_(programming_language)> rdfs:label ?label ;
                                                                       dbo:abstract ?abstract . 
            FILTER (lang(?label) = 'en' && lang(?abstract) = 'en')
        }
        LIMIT 10
        """,
        "Countries and capitals": """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?country ?capital
        WHERE {
            ?country a dbo:Country ;
                     dbo:capital ?capital . 
            ?country rdfs:label ?countryLabel . 
            ?capital rdfs:label ?capitalLabel . 
            FILTER (lang(?countryLabel) = 'en' && lang(?capitalLabel) = 'en')
        }
        LIMIT 10
        """
    }

    # Button to execute selected query
    execute_button = tk.Button(frame, text="Execute Query", command=lambda: on_execute_query(selected_query.get(), queries[selected_query.get()], result_display))
    execute_button.pack(pady=10)

    # Section to add new queries
    add_query_frame = tk.Frame(root)
    add_query_frame.pack(pady=10)

    query_name_label = tk.Label(add_query_frame, text="Query Name:")
    query_name_label.grid(row=0, column=0)
    query_name_entry = tk.Entry(add_query_frame)
    query_name_entry.grid(row=0, column=1)

    query_text_label = tk.Label(add_query_frame, text="Query Text:")
    query_text_label.grid(row=1, column=0)
    query_text_entry = tk.Text(add_query_frame, width=50, height=5)
    query_text_entry.grid(row=1, column=1)

    add_query_button = tk.Button(add_query_frame, text="Add Query", command=lambda: on_add_query(query_name_entry, query_text_entry, queries, query_menu))
    add_query_button.grid(row=2, columnspan=2, pady=5)

    # Button to delete selected query
    delete_query_button = tk.Button(frame, text="Delete Selected Query", command=lambda: on_delete_query(selected_query, queries, query_menu))
    delete_query_button.pack(pady=5)

    # Start the GUI
    root.mainloop()

if __name__ == "__main__":
    create_gui()
